# Testing OpenCV algorithms

In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [12]:
# Load the pathimage
path = 'Data/retinal-lesions-v20191227/images_896x896'

for idx, imp in enumerate(os.listdir(path)[11:18]):
    im_path = os.path.join(path, imp)
    orig_img = cv2.imread(im_path)
    img = orig_img.copy()

    # Extract the green plane
    green_plane = img[:, :, 1]
    green_orig = green_plane.copy()

    cv2.imshow('Original Image', orig_img)
    cv2.waitKey(0)
    cv2.imshow('Green Plane Image', green_orig)
    cv2.waitKey(0)

    for t in [22,24,27]:
        for c in [4.5, 5]:
            # Define the FA-CLAHE algorithm
            green_plane = green_orig.copy()
            faclahe = cv2.createCLAHE(clipLimit=c, tileGridSize=(t, t))

            # Apply the FA-CLAHE algorithm to the green plane
            faclahe_img = faclahe.apply(green_plane)

            # Replace the green plane with the enhanced green plane
            img[:, :, 1] = faclahe_img

            # Display the original and enhanced images
            cv2.imshow(f'FA-CLAHE Enhanced Image {t}-{c}', faclahe_img)
            cv2.waitKey(0)
        
    cv2.destroyAllWindows()

In [4]:
im_path = 'Data/retinal-lesions-v20191227/lesion_segs_896x896/original/9_left/cotton_wool_spots.png'
# Load the input image in grayscale mode
img = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE)

# Apply binary thresholding to convert the image to a binary image
ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Get the connected components of the binary image
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

# Iterate over the connected components and display them one by one
for i in range(1, num_labels):
    # Create a binary image for the current component
    component = (labels == i).astype(np.uint8) * 255
    
    # Display the current component
    cv2.imshow('Component {}'.format(i), component)
    
    # Wait for a key event
    cv2.waitKey(0)

    # Close all open windows
    cv2.destroyAllWindows()


In [3]:
os.path.exists('mask.png')

False